## Import libs, set paths and load params

In [1]:
import os
import numpy as np
import pandas as pd
import sys
sys.path.insert(0, "../src")

import auxilary_functions as f

## Convert raw data to adjacency matrices

In [2]:
#ecoli
df = f.read_ecoli_network("../data/ori/ecoli/network_tf_gene.txt")
edges = df[[1, 3]]
edges.columns = ["tf", "tg"]
edges.tf = edges.tf.apply(lambda x: x.lower())
edges.tg = edges.tg.apply(lambda x: x.lower())
nodes = sorted(np.unique(np.concatenate((edges.tf.unique(), edges.tg.unique()))))
nodes = pd.DataFrame(data=range(len(nodes)), index=nodes, columns=["idx"])
edges_ = edges.join(nodes, on="tf").join(nodes, on="tg", lsuffix="_tf", rsuffix="_tg")
np_edges = edges_[["idx_tg", "idx_tf"]].values
interaction_matrix_ecoli = f.build_Tnet(np_edges, len(nodes))

In [36]:
import networkx as nx
interaction_matrix_graph = nx.DiGraph(interaction_matrix_human)
degree_list_test = sorted([d for n, d in interaction_matrix_graph.out_degree], reverse=True)

counts = {item:degree_list_test.count(item) for item in degree_list_test}

for i in range(20):
    print(f"number of genes with {i} regulators: {counts.get(i)}")

number of genes with 0 regulators: 370
number of genes with 1 regulators: 1075
number of genes with 2 regulators: 507
number of genes with 3 regulators: 271
number of genes with 4 regulators: 181
number of genes with 5 regulators: 121
number of genes with 6 regulators: 57
number of genes with 7 regulators: 57
number of genes with 8 regulators: 39
number of genes with 9 regulators: 28
number of genes with 10 regulators: 23
number of genes with 11 regulators: 23
number of genes with 12 regulators: 20
number of genes with 13 regulators: 10
number of genes with 14 regulators: 11
number of genes with 15 regulators: 10
number of genes with 16 regulators: 7
number of genes with 17 regulators: 5
number of genes with 18 regulators: 2
number of genes with 19 regulators: 3


In [30]:
#yeast
edges = pd.read_csv("../data/ori/yeast/tnet.txt", sep="\t")
nodes = sorted(np.unique(np.concatenate((edges.Tf.unique(), edges.Tg.unique()))))
nodes = pd.DataFrame(data=range(len(nodes)), index=nodes, columns=["idx"])
edges_ = edges.join(nodes, on="Tf").join(nodes, on="Tg", lsuffix="_tf", rsuffix="_tg")
np_edges = edges_[["idx_tg", "idx_tf"]].values
interaction_matrix_yeast = f.build_Tnet(np_edges, len(nodes))

In [31]:
#mouse
edges = pd.read_csv(("../data/ori/trrust/trrust_rawdata.mouse.tsv"), sep="\t", header=None)[[0, 1]]
edges.columns = ["Tf", "Tg"]
nodes = sorted(np.unique(np.concatenate((edges.Tf.unique(), edges.Tg.unique()))))
nodes = pd.DataFrame(data=range(len(nodes)), index=nodes, columns=["idx"])
edges_ = edges.join(nodes, on="Tf").join(nodes, on="Tg", lsuffix="_tf", rsuffix="_tg")
np_edges = edges_[["idx_tg", "idx_tf"]].values
interaction_matrix_mouse = f.build_Tnet(np_edges, len(nodes))

In [32]:
#human
edges = pd.read_csv(("../data/ori/trrust/trrust_rawdata.human.tsv"), sep="\t", header=None)[[0, 1]]
edges.columns = ["Tf", "Tg"]
nodes = sorted(np.unique(np.concatenate((edges.Tf.unique(), edges.Tg.unique()))))
nodes = pd.DataFrame(data=range(len(nodes)), index=nodes, columns=["idx"])
edges_ = edges.join(nodes, on="Tf").join(nodes, on="Tg", lsuffix="_tf", rsuffix="_tg")
np_edges = edges_[["idx_tg", "idx_tf"]].values
interaction_matrix_human = f.build_Tnet(np_edges, len(nodes))

## Create table

In [6]:
cfg = f.get_actual_parametrization("../src/config.json")
table1 = pd.DataFrame([f.collect_topological_parameters(cfg,interaction_matrix_ecoli, 'e.coli'),
                      f.collect_topological_parameters(cfg,interaction_matrix_yeast, 'yeast'),
                      f.collect_topological_parameters(cfg,interaction_matrix_mouse, 'mouse'),
                      f.collect_topological_parameters(cfg,interaction_matrix_human, 'human')], 
                  columns = ['ffl_node_participation' , 'sparsity', 'in-degree' , 'out-degree', 'organism']) 

table1.to_csv("../results/tables/table1.tsv", sep="\t", index=False)
table1

,ffl_node_participation,sparsity,in-degree,out-degree,organism
0,1392,2.328,1.106,1.222,e.coli
1,3370,2.899,1.421,1.477,yeast
2,1956,2.643,1.274,1.369,mouse
3,4152,2.944,1.364,1.580,human
